In [1]:
##Cormac McHale

In [2]:
##Training the Neural Network

In [3]:
import gzip
import sklearn.preprocessing as pre
import keras as kr
import numpy as np
from keras.models import load_model

Using TensorFlow backend.


In [4]:
# Start a neural network, building it by layers.
model = kr.models.Sequential()

# Add a hidden layer with 1000 neurons and an input layer with 784.
model.add(kr.layers.Dense(units=1000, activation='relu', input_dim=784))
model.add(kr.layers.Dense(units=100, activation='sigmoid'))
model.add(kr.layers.Dense(units=1000, activation='relu'))
# Add a 10 neuron output layer for each digit??
model.add(kr.layers.Dense(units=10, activation='softmax'))

# Build the graph.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
##file reading and formatting
imagesFile  = open('MNIST/train-images.idx3-ubyte', 'rb')
labelsFile = open('MNIST/train-labels.idx1-ubyte', 'rb')
images = imagesFile.read()
labels = labelsFile.read()
images =  (~np.array(list(images[16:])).reshape(60000, 28, 28).astype(np.uint8)-255)
labels =  np.array(list(labels[ 8:])).astype(np.uint8)

In [6]:
inputs = images.reshape(60000, 784)
inputs[inputs>0] = 40
inputs[inputs<1] = 0 

In [7]:
encoder = pre.LabelBinarizer()
encoder.fit(labels)
outputs = encoder.transform(labels)
#just for looking
#print(inputs[0])
#print (inputs[0].size)

In [8]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1000)              785000    
_________________________________________________________________
dense_2 (Dense)              (None, 100)               100100    
_________________________________________________________________
dense_3 (Dense)              (None, 1000)              101000    
_________________________________________________________________
dense_4 (Dense)              (None, 10)                10010     
Total params: 996,110
Trainable params: 996,110
Non-trainable params: 0
_________________________________________________________________


In [9]:
##change later to test
model.fit(inputs, outputs, epochs=5, batch_size=150)

Epoch 1/5
60000/60000 [==============================] - 11s 189us/step - loss: 0.3825 - accuracy: 0.8871
Epoch 2/5
60000/60000 [==============================] - 11s 178us/step - loss: 0.2202 - accuracy: 0.9326
Epoch 3/5
60000/60000 [==============================] - 11s 183us/step - loss: 0.1782 - accuracy: 0.9447
Epoch 4/5
60000/60000 [==============================] - 11s 179us/step - loss: 0.1421 - accuracy: 0.9569
Epoch 5/5
60000/60000 [==============================] - 11s 181us/step - loss: 0.1310 - accuracy: 0.9600


In [10]:
##need to then use the test images on the Neural Network...
imagesTest  = open('MNIST/t10k-images.idx3-ubyte', 'rb')
labelsTest = open('MNIST/t10k-labels.idx1-ubyte', 'rb')
testImages = imagesTest.read()
testLabels = labelsTest.read()

In [11]:
type(testImages)

bytes

In [12]:
testImages[0:4]

b'\x00\x00\x08\x03'

In [13]:
testImages = (~np.array(list(testImages[16:])).reshape(10000, 28, 28).astype(np.uint8)-255)
testLabels =  np.array(list(testLabels[ 8:])).astype(np.uint8)
testInputs = testImages.reshape(10000, 784)
encoder = pre.LabelBinarizer()
encoder.fit(testLabels)
testOutputs = encoder.transform(testLabels)

In [14]:
# Have the network predict the classes of the test inputs.
predictions = model.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

7196

In [15]:
#save the model
model.save('savedModel/my_model.h5')
##Initial Model finished... flask Next.. will return to this later
testInputs = testImages.reshape(10000, 784)
model.predict(testInputs)

array([[2.27964529e-06, 1.89340572e-05, 2.06188088e-05, ...,
        9.99869108e-01, 7.54411758e-07, 7.32124099e-05],
       [9.23021215e-09, 1.64814980e-03, 9.98258412e-01, ...,
        3.79524608e-05, 8.89944022e-06, 1.89586480e-08],
       [1.23481811e-06, 9.99602139e-01, 1.25902316e-05, ...,
        2.06011773e-05, 2.80652923e-04, 5.46228875e-06],
       ...,
       [1.75650388e-01, 1.15715840e-04, 1.99300447e-03, ...,
        1.13449795e-02, 1.71350874e-02, 7.77535558e-01],
       [2.66626070e-04, 3.81777133e-03, 5.20698195e-05, ...,
        4.33714558e-05, 2.96349004e-02, 7.19612115e-04],
       [7.25531281e-05, 4.71758004e-03, 9.59660769e-01, ...,
        2.05620599e-04, 1.00115594e-02, 6.43830754e-06]], dtype=float32)

In [16]:
##Model complete

In [17]:
##Re-train with an extra layer

In [18]:
type(model)

keras.engine.sequential.Sequential

In [19]:
fineTune = kr.models.Sequential()
for layer in model.layers:
    fineTune.add(layer)
for layer in fineTune.layers:
    layer.trainable = False

In [20]:
fineTune.add(kr.layers.Dense(units=10, activation='sigmoid'))

In [21]:
inputs[inputs>0] = 1
testInputs[testInputs>0] = 1

In [22]:
fineTune.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
fineTune.fit(inputs, outputs, epochs=10, batch_size=50)

Epoch 1/10
60000/60000 [==============================] - 5s 77us/step - loss: 0.3942 - accuracy: 0.9031
Epoch 2/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.1604 - accuracy: 0.9845
Epoch 3/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0927 - accuracy: 0.9901
Epoch 4/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0620 - accuracy: 0.9912
Epoch 5/10
60000/60000 [==============================] - 5s 76us/step - loss: 0.0457 - accuracy: 0.9918
Epoch 6/10
60000/60000 [==============================] - 4s 70us/step - loss: 0.0364 - accuracy: 0.9922
Epoch 7/10
60000/60000 [==============================] - 4s 68us/step - loss: 0.0310 - accuracy: 0.9925
Epoch 8/10
60000/60000 [==============================] - 4s 70us/step - loss: 0.0278 - accuracy: 0.9927
Epoch 9/10
60000/60000 [==============================] - 4s 70us/step - loss: 0.0260 - accuracy: 0.9929
Epoch 10/10
60000/60000 [==============================

In [23]:
# Have the network predict the classes of the test inputs.
predictions = fineTune.predict(testInputs)
predictions = encoder.inverse_transform(predictions)
##Comparison
predictions == encoder.inverse_transform(testOutputs)
#check how 
(predictions == encoder.inverse_transform(testOutputs)).sum()

9596

In [24]:
#Results
#save new model
fineTune.save('savedModel/freshlyTrained.h5')
#test in client